In [1]:
from helper import *
print("col:", col, "row:", row)

col: 9 row: 6


In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((row*col,3), np.float32)
objp[:,:2] = np.mgrid[0:col, 0:row].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (col,row), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (col,row), corners, ret)
        #write_name = 'corners_found'+str(idx)+'.jpg'
        #cv2.imwrite(write_name, img)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [5]:
# transform matrix
src = np.float32([
    [700, 447], # up right
    [1305, 720], # down right
    [6, 720], # down left
    [580, 447] # up left
])
dst = np.float32(
    [
        [1180, 0],
        [1180, 720],
        [100, 720],
        [100, 0],
    ])

In [6]:
perspective_M, invert_perspective_M = cv2.getPerspectiveTransform(src, dst), cv2.getPerspectiveTransform(dst, src)
img_size = (1280, 720)
# Do camera calibration given object points and image points
# mtx is the calibration matrix
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    
# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["src"] = src
dist_pickle["dst"] = dst
dist_pickle["objpoints"] = objpoints
dist_pickle["imgpoints"] = imgpoints
dist_pickle["perspective_M"] = perspective_M
dist_pickle["invert_perspective_M"] = invert_perspective_M
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "wide_dist_pickle.p", "wb" ) )